## Imports 

In [3]:
import requests 
from bs4 import BeautifulSoup 
import json
import time
from urllib.parse import urljoin


## WEB CRAWLER
Ένας crawler (ανιχνευτής) που περιηγείται τη Wikipedia, συλλέγει άρθρα και τα αποθηκεύει σε ένα αρχείο JSON. 
WikipediaCrawler:
Η κλάση είναι υπεύθυνη για την αναζήτηση, την επεξεργασία, και την αποθήκευση άρθρων από τη Wikipedia.
base_url: Η βασική διεύθυνση URL της Wikipedia (π.χ., "https://en.wikipedia.org").
visited: Ένα σύνολο URLs που έχουν ήδη επισκεφθεί για αποφυγή διπλών επισκέψεων.
max_articles: Το μέγιστο πλήθος άρθρων που θα συλλέξει ο crawler.
articles: Λίστα με τα άρθρα που θα αποθηκευτούν.
keywords: Λέξεις-κλειδιά που καθορίζουν ποια άρθρα θα συλλέγονται. Αν δεν δοθούν, συλλέγονται όλα τα άρθρα.

Μέθοδος: fetch_page
Σκοπός: Λήψη HTML περιεχομένου από μια διεύθυνση URL.
Χρησιμοποιεί τη βιβλιοθήκη requests για να κάνει το αίτημα HTTP.
Αν το αίτημα είναι επιτυχές (status code 200), επιστρέφεται το HTML. Αν όχι, εκτυπώνεται μήνυμα σφάλματος.

Μέθοδος: parse_article
Σκοπός: Εξαγωγή του τίτλου και του περιεχομένου ενός άρθρου.
Χρησιμοποιεί το BeautifulSoup για parsing του HTML.
Εντοπίζει τον τίτλο του άρθρου (μέσα στην ετικέτα h1 με id="firstHeading") και το περιεχόμενο (μέσα σε παραγράφους p).
Φιλτράρει τα άρθρα: Αν έχουν καθοριστεί λέξεις-κλειδιά, ελέγχει αν το περιεχόμενο περιέχει μία από αυτές τις λέξεις. Αν όχι, απορρίπτεται το άρθρο.

Μέθοδος: crawl
Σκοπός: Ανίχνευση της Wikipedia για συλλογή άρθρων.
Ξεκινά από ένα αρχικό μονοπάτι (π.χ., /wiki/Web_scraping).
Χρησιμοποιεί έναν queue-based αλγόριθμο (παρόμοιο με BFS) για να ακολουθεί τα links σε κάθε άρθρο.
Φιλτράρει:
Links που δεν ξεκινούν με /wiki/.
Links με ειδικούς χαρακτήρες (π.χ., : που δηλώνει κατηγορίες ή αρχεία).
Διπλότυπα URLs χρησιμοποιώντας το σύνολο self.visited.
Διαλείμματα: Περιμένει 1 δευτερόλεπτο μεταξύ αιτημάτων για να αποφύγει υπερφόρτωση του server.



In [4]:

class WikipediaCrawler:
    def __init__(self, base_url="https://en.wikipedia.org", max_articles=100, keywords=None):
        self.base_url = base_url
        self.visited = set()
        self.max_articles = max_articles
        self.articles = []
        self.keywords = keywords if keywords else []

    def fetch_page(self, url):
        try:
            response = requests.get(url, timeout=2.0)
            if response.status_code == 200:
                return response.text
            else:
                print(f"Failed to fetch {url}, status code: {response.status_code}")
        except Exception as e:
            print(f"Error fetching {url}: {e}")
        return None

    def parse_article(self, url):
        html_content = self.fetch_page(url)
        if html_content is None:
            return None

        soup = BeautifulSoup(html_content, "html.parser")
        title_tag = soup.find("h1", id="firstHeading")
        content = " ".join([p.text for p in soup.find_all("p")])

        if not title_tag or not content.strip():
            return None  # Skip articles without title or content

        title = title_tag.text.strip()

        # Check if article content matches keywords
        if self.keywords and not any(keyword.lower() in content.lower() for keyword in self.keywords):
            return None

        return {"title": title, "url": url, "content": content}

    def crawl(self, start_path="/wiki/Web_scraping"):
        queue = [start_path]
        
        while queue and len(self.articles) < self.max_articles:
            path = queue.pop(0)
            full_url = urljoin(self.base_url, path)

            if full_url in self.visited:
                continue

            self.visited.add(full_url)
            article = self.parse_article(full_url)
            if article:
                print(f"Crawled: {article['title']} ({len(self.articles) + 1}/{self.max_articles})")
                self.articles.append(article)

                # Extract links to other articles
                soup = BeautifulSoup(self.fetch_page(full_url), "html.parser")
                for link in soup.find_all("a", href=True):
                    href = link["href"]
                    if href.startswith("/wiki/") and ":" not in href and href not in self.visited:
                        queue.append(href)
            
            # Avoid overwhelming the server
            time.sleep(1)

        print(f"Crawling completed. Collected {len(self.articles)} articles.")

    def save_to_file(self, file_name="filtered_wikipedia_articles.json"):
        with open(file_name, "w", encoding="utf-8") as file:
            json.dump(self.articles, file, indent=4, ensure_ascii=False)
        print(f"Data saved to {file_name}")


if __name__ == "__main__":
    # Define keywords for filtering articles
    keywords = ["machine learning", "artificial intelligence", "neural networks","information retrieval","programming","database","algorithms","algorithm"]
    crawler = WikipediaCrawler(max_articles=100, keywords=keywords)  # Adjust max_articles as needed
    crawler.crawl(start_path="/wiki/Artificial_intelligence")
    crawler.save_to_file()

Crawled: Artificial intelligence (1/100)
Crawled: Ai (2/100)
Crawled: Artificial intelligence (disambiguation) (3/100)
Crawled: Artificial general intelligence (4/100)
Crawled: Intelligent agent (5/100)
Crawled: Automated planning and scheduling (6/100)
Crawled: Computer vision (7/100)
Crawled: General game playing (8/100)
Crawled: Knowledge representation and reasoning (9/100)
Crawled: Natural language processing (10/100)
Crawled: Robotics (11/100)
Crawled: AI safety (12/100)
Crawled: Machine learning (13/100)
Crawled: Symbolic artificial intelligence (14/100)
Crawled: Deep learning (15/100)
Crawled: Bayesian network (16/100)
Crawled: Evolutionary algorithm (17/100)
Crawled: Hybrid intelligent system (18/100)
Crawled: Artificial intelligence systems integration (19/100)
Crawled: Applications of artificial intelligence (20/100)
Crawled: Machine learning in bioinformatics (21/100)
Crawled: Deepfake (22/100)
Crawled: Machine learning in earth sciences (23/100)
Crawled: Applications of ar